<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Modelo de lenguaje con tokenización por caracteres

### Entrenamiento

El entranamiento se hizo utilizando el script "train.py" ver `index.md` para ver resultados del entranmiento y cono se realizo el mismo


### Generación de secuencias

In [1]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import pad_sequences
import os

2025-12-10 22:37:12.417971: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
def hidratar_diccionarios(vocab_path='vocab.pkl'):
    """
    Carga los diccionarios, inyectándolos en el entorno para
    que las funciones de inferencia puedan usarlos directamente.
    """
    if not os.path.exists(vocab_path):
        print(f"⚠️ ERROR: No se encuentran {vocab_path}")
        print("Asegúrate de subir los archivos generados por train.py al entorno del notebook.")
        return None, None, None

    print(f"Cargando vocabulario desde {vocab_path}...")
    with open(vocab_path, 'rb') as f:
        vocab_data = pickle.load(f)

    char2idx = vocab_data['char2idx']
    idx2char = vocab_data['idx2char']

    print("✅ ¡Entorno hidratado correctamente!")
    return char2idx, idx2char




In [20]:
# --- EJECUTAR CARGA ---
# Esto define las variables globales char2idx e idx2char
char2idx, idx2char = hidratar_diccionarios()
model_simplernn = load_model('simplernn_best_model.keras')
model_gru = load_model('gru_best_model.keras')
model_lstm = load_model('lstm_best_model.keras')

Cargando vocabulario desde vocab.pkl...
✅ ¡Entorno hidratado correctamente!


In [16]:
def generate_seq(model, seed_text, max_length, n_words):
    """
        Exec model sequence prediction

        Args:
            model (keras): modelo entrenado
            seed_text (string): texto de entrada (input_seq)
            max_length (int): máxima longitud de la sequencia de entrada
            n_words (int): números de caracteres a agregar a la sequencia de entrada
        returns:
            output_text (string): sentencia con las "n_words" agregadas
    """
    output_text = seed_text
	# generate a fixed number of words
    for _ in range(n_words):
		# Encodeamos
        encoded = [char2idx[ch] for ch in output_text.lower() ]
		# Si tienen distinto largo
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')

		# Predicción softmax
        y_hat = np.argmax(model.predict(encoded,verbose=0)[0,-1,:])
		# Vamos concatenando las predicciones
        out_word = ''

        out_word = idx2char[y_hat]

		# Agrego las palabras a la frase predicha
        output_text += out_word
    return output_text

In [ ]:
# Definimos los modelos en un diccionario
modelos = {
    "SimpleRNN": model_simplernn,
    "GRU": model_gru,
    "LSTM": model_lstm
}


In [27]:
seed = 'habia una vez'
n_words = 100
max_length = 100

print(f"--- SEMILLA: '{seed}' ---\n")

for nombre, modelo in modelos.items():
    print(f"🔵 Modelo: {nombre}")
    texto = generate_seq(modelo, seed, max_length, n_words)
    print(f"Generado: ...{texto[len(seed):]}")
    print(f"Full:     {texto}")
    print("-" * 50) # Separador visual

--- SEMILLA: 'habia una vez' ---

🔵 Modelo: SimpleRNN
Generado: ... en el camino de la carca de la carca de la carca de la carca de la carca de la carca de la carca de
Full:     habia una vez en el camino de la carca de la carca de la carca de la carca de la carca de la carca de la carca de
--------------------------------------------------
🔵 Modelo: GRU
Generado: ... en el camino de la mañana, se había después de haber ser 
visto a la estación de la mañana, se habí
Full:     habia una vez en el camino de la mañana, se había después de haber ser 
visto a la estación de la mañana, se habí
--------------------------------------------------
🔵 Modelo: LSTM
Generado: ... en el camino de la mañana, el tren se había desaparecido en 
consiguiente, el tren se había desapar
Full:     habia una vez en el camino de la mañana, el tren se había desaparecido en 
consiguiente, el tren se había desapar
--------------------------------------------------


SimpleRNN entró en loop como era de esperar (memoria corta) y GRU y LSTM aprendio algo más sobre el libro.

###  Beam search y muestreo aleatorio

In [6]:
# funcionalidades para hacer encoding y decoding

def encode(text, max_length=100):

    encoded = [char2idx[ch] for ch in text]
    encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')

    return encoded

def decode(seq):
    return ''.join([idx2char[ch] for ch in seq])

In [7]:
from scipy.special import softmax

# función que selecciona candidatos para el beam search
def select_candidates(pred,num_beams,vocab_size,history_probs,history_tokens,temp,mode):

  # colectar todas las probabilidades para la siguiente búsqueda
  pred_large = []

  for idx,pp in enumerate(pred):
    pred_large.extend(np.log(pp+1E-10)+history_probs[idx])

  pred_large = np.array(pred_large)

  # criterio de selección
  if mode == 'det':
    idx_select = np.argsort(pred_large)[::-1][:num_beams] # beam search determinista
  elif mode == 'sto':
    idx_select = np.random.choice(np.arange(pred_large.shape[0]), num_beams, p=softmax(pred_large/temp)) # beam search con muestreo aleatorio
  else:
    raise ValueError(f'Wrong selection mode. {mode} was given. det and sto are supported.')

  # traducir a índices de token en el vocabulario
  new_history_tokens = np.concatenate((np.array(history_tokens)[idx_select//vocab_size],
                        np.array([idx_select%vocab_size]).T),
                      axis=1)

  # devolver el producto de las probabilidades (log) y la secuencia de tokens seleccionados
  return pred_large[idx_select.astype(int)], new_history_tokens.astype(int)


def beam_search(model,num_beams,num_words,input,temp=1,mode='det'):

    # first iteration

    # encode
    encoded = encode(input)

    # first prediction
    y_hat = model.predict(encoded,verbose=0)[0,-1,:]

    # get vocabulary size
    vocab_size = y_hat.shape[0]

    # initialize history
    history_probs = [0]*num_beams
    history_tokens = [encoded[0]]*num_beams

    # select num_beams candidates
    history_probs, history_tokens = select_candidates([y_hat],
                                        num_beams,
                                        vocab_size,
                                        history_probs,
                                        history_tokens,
                                        temp,
                                        mode)

    # beam search loop
    for i in range(num_words-1):

      preds = []

      for hist in history_tokens:

        # actualizar secuencia de tokens
        input_update = np.array([hist[i+1:]]).copy()

        # predicción
        y_hat = model.predict(input_update,verbose=0)[0,-1,:]

        preds.append(y_hat)

      history_probs, history_tokens = select_candidates(preds,
                                                        num_beams,
                                                        vocab_size,
                                                        history_probs,
                                                        history_tokens,
                                                        temp,
                                                        mode)

    return history_tokens[:,-(len(input)+num_words):]

In [40]:
# Configuración global
seed = "habia una vez"
L = 50  # Longitud a generar
num_beams_det = 5   # Beams para determinista
num_beams_sto = 10  # Beams para estocástico (más ancho ayuda a la variedad)
temperaturas = [0.2, 0.5, 1.0, 1.2, 1.5]

# Definimos tus modelos disponibles
# Asegúrate de tener estas variables ya cargadas en tu notebook
modelos = {
    "SimpleRNN": model_simplernn,
    "GRU": model_gru,
    "LSTM": model_lstm
}

def evaluar_generacion(nombre_modelo, modelo):
    print(f"\n{'='*60}")
    print(f"🤖 EVALUANDO MODELO: {nombre_modelo}")
    print(f"{'='*60}")

    # --- 1. Beam Search Determinista ---
    print(f"\n🔹 Modo: Beam Search Determinista (Beams={num_beams_det})")
    try:
        secuencias = beam_search(
            model=modelo,
            num_beams=num_beams_det,
            num_words=L,
            input=seed,
            mode='det'
        )
        # En determinista, el índice 0 es el de mayor probabilidad acumulada
        texto = decode(secuencias[0])
        print(f"   Generado: ...{texto}")
    except Exception as e:
        print(f"   ❌ Error: {e}")

    # --- 2. Beam Search Estocástico con Temperaturas ---
    print(f"\n🔹 Modo: Beam Search Estocástico")

    for temp in temperaturas:
        try:
            secuencias = beam_search(
                model=modelo,
                num_beams=num_beams_sto,
                num_words=L,
                input=seed,
                temp=temp,
                mode='sto'
            )

            # En estocástico, cualquiera de los caminos es válido. Mostramos el primero.
            texto = decode(secuencias[0])
            print(f"   🌡️ Temp {temp}: ...{texto}")

        except Exception as e:
            print(f"   ❌ Error con temp {temp}: {e}")

# --- Bucle Principal ---
for nombre, modelo in modelos.items():
    evaluar_generacion(nombre, modelo)


🤖 EVALUANDO MODELO: SimpleRNN

🔹 Modo: Beam Search Determinista (Beams=5)
   Generado: ...habia una vez en su compañero de los viajeros que se había comp

🔹 Modo: Beam Search Estocástico
   🌡️ Temp 0.2: ...habia una vez en el conductor de la mañana, sin embargo, el tre
   🌡️ Temp 0.5: ...habia una vez en el capitán de la estación de la estación de la
   🌡️ Temp 1.0: ...habia una vez en el vapor de esta del respondió picaporte se 
v
   🌡️ Temp 1.2: ...habia una vez en la 
compañía del capitán de la compañía de la 
   🌡️ Temp 1.5: ...habia una vez en el puerto de la calcular del vapor de toda vis

🤖 EVALUANDO MODELO: GRU

🔹 Modo: Beam Search Determinista (Beams=5)
   Generado: ...habia una vez en la estación de la estación de la estación del 

🔹 Modo: Beam Search Estocástico
   🌡️ Temp 0.2: ...habia una vez en el camino de la mañana, se había después de ha
   🌡️ Temp 0.5: ...habia una vez en la estación de la estación de los viajeros que
   🌡️ Temp 1.0: ...habia una vez en su casa de lo

Arquitectura: La LSTM es la más robusta para este dataset, logrando capturar nombres propios y estructura sintáctica.

Estrategia de Generación: El Beam Search Estocástico con una temperatura cercana a 1.0 produce los textos más naturales, evitando los bucles repetitivos del determinista.

